<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/inference_feat/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Detection ML Test

## 1. Prerequisites

This section provides the setting up of the environment

In [1]:
!pip install torch -f https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchaudio -f https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchsummary==1.5.1 torchtext==0.13.1 torchvision -f https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl

Looking in links: https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
Looking in links: https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
Looking in links: https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl


In [2]:
!git clone https://github.com/v-iashin/video_features.git
!pip install omegaconf==2.0.6

fatal: destination path 'video_features' already exists and is not an empty directory.


fatal: destination path 'ict3104-p2-team08' already exists and is not an empty directory.


In [3]:
# Imports widgets for jupyter
try:
    import piplite
    piplite.install(['ipywidgets'])
except ImportError:
    pass

## 2. Toggling of pipelines

This section shows the toggling of dependencies between TSU and STEP

In [32]:
#insert code here

C:\Users\leech\ict3104-p2-team08


## 3. Data exploration

This section load and display video data from the Toyota Smarthome (TSU) project

In [6]:
#import tsu video files
import os
FOLDER_PATH = 'input_files'
ROOT_PATH = '/content/data'

file_list = os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/data\\input_files'

In [8]:
#tsu video files to display in dropdown
from ipywidgets import  Dropdown, widgets

w = widgets.Dropdown(
    options= file_list,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

w.observe(on_change)

display(w)

NameError: name 'file_list' is not defined

In [9]:
#display video 
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/data/input_files/%' + w.value

locate= "" 

for i in video_path: 
   if i=='%': 
       pass
   else: 
       locate+=i 

def show_video(locate, video_width = 500):
  video_file = open(locate, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
  
show_video(locate)

NameError: name 'w' is not defined

In [ ]:
#only run it for the first time
%cd video_features

In [ ]:
#extract to I3D feature file
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

# Select the feature type
feature_type = 'i3d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/data/input_files/P02T02C06.mp4']
args.flow_type = 'raft'

extractor = ExtractI3D(args)
print(locate)

# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

## 4. Inference with pretrained model

This section allows generation of caption videos

In [6]:
import os
print(os.getcwd())

C:\Users\leech\ict3104-p2-team08


### Select a pre-trained model

In [1]:
#tsu video files to display in dropdown
import ipywidgets as widgets
import os

MODEL_PATH = 'models'
MODEL_ROOT_PATH = './Toyota_Smarthome/pipline/'

file_list = os.listdir(os.path.join(MODEL_ROOT_PATH, MODEL_PATH))

model_dropdown = widgets.Dropdown(
    options=file_list,
    description='Model:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

model_dropdown.observe(on_change)
display(model_dropdown)

Dropdown(description='Model:', options=('PDAN',), value='PDAN')

### Choose an input video

In [2]:
VIDEO_PATH = 'input_files'
VIDEO_ROOT_PATH = './data/'

file_list = os.listdir(os.path.join(VIDEO_ROOT_PATH, VIDEO_PATH))

video_dropdown = widgets.Dropdown(
    options=file_list,
    description='Video:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

video_dropdown.observe(on_change)
display(video_dropdown)

Dropdown(description='Video:', options=('P02T01C06.mp4', 'P02T01C07.mp4', 'P02T02C03.mp4', 'P02T02C06.mp4', 'P…

### Generate video with caption

In [3]:
#insert codes here
from IPython.display import Video

video_selected = video_dropdown.value.replace('.mp4','')
model_selected = model_dropdown.value
%run -i ./Toyota_Smarthome/pipline/test2.py -videofile={video_selected} -model={model_selected}
Video("./Toyota_Smarthome/pipline/video_output/" + video_selected + "_caption.mp4")

100%|███████████████████████████████████████████████████████████████████████████████| 536/536 [00:04<00:00, 108.08it/s]


51


generating video...: 100%|███████████████████████████████████████████████████████| 17769/17769 [04:37<00:00, 64.23it/s]

generating video...: : 17770it [04:53, 64.23it/s]                                                                      

## 5. Testing and evaluating trained models

This section shows

## 6. Training of new model

This section shows